In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [10]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear= nn.Linear(16,384)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 4),
#             nn.Softmax()
        )
        
        self.idx = nn.Embedding(num_embeddings=3, 
                               embedding_dim=128,
                               padding_idx=1)

    def forward(self, x):
        data, idx = x[:, :-1], x[:,-1:]
        
        idx_emb = self.idx(idx.int()).flatten(1)
        data = self.linear(data)
        x = torch.cat([data,idx_emb],1)
                
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
from torch.optim import Adam

model = NeuralNetwork().to(device)
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [12]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader

class CustomImageDataset(Dataset):
    def __init__(self, is_train=True, transform=None, target_transform=None):
        self.is_train = is_train
        if self.is_train:
            df1 = pd.read_csv("no_csv.csv",).drop(["Unnamed: 0", "inter","date"], axis = 'columns')[:-3]
            df2 = pd.read_csv("son_csv.csv").drop(["Unnamed: 0", "inter","date"], axis = 'columns')[:-3]
            df3 = pd.read_csv("cho_csv.csv").drop(["Unnamed: 0", "inter","date"], axis = 'columns')[:-3]
            
        else:
            df1 = pd.read_csv("no_csv.csv",).drop(["Unnamed: 0", "inter","date"], axis = 'columns')[-3:]
            df2 = pd.read_csv("son_csv.csv").drop(["Unnamed: 0", "inter","date"], axis = 'columns')[-3:]
            df3 = pd.read_csv("cho_csv.csv").drop(["Unnamed: 0", "inter","date"], axis = 'columns')[-3:]

        df1["index"] = 0
        df2["index"] = 1
        df3["index"] = 2

        df = pd.concat([df1,df2,df3])

        ans = df['target'].to_numpy() - 1 
        data = df.drop("target", axis = 'columns').to_numpy()

        self.ans = torch.nn.functional.one_hot(torch.from_numpy(ans).to(torch.int64), num_classes=4)
        self.data = torch.from_numpy(data).float()
        
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.ans[idx]
        

In [13]:
trainset = CustomImageDataset(is_train=True)
train_loader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)


testset = CustomImageDataset(is_train=False)
test_loader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

In [14]:
for i, (data, labels) in enumerate(train_loader, 0):
    break
    
criterion = nn.CrossEntropyLoss()

In [31]:
def train(num_epochs):
    best_accuracy = 0
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, (data, labels) in enumerate(train_loader, 0):
            
            # get the inputs
            data = data.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            outputs = model(data)

            loss = criterion(outputs,labels.float().cuda())
            loss.backward()
            optimizer.step()

            # Let's print statistics for every 1,000 images
            running_loss += loss.item()     # extract the loss value
            if i % 1000 == 999:    
                # print every 1000 (twice per epoch) 
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                # zero the loss
                running_loss = 0.0
#             print(loss)
        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
#             saveModel()
            best_accuracy = accuracy
            
def testAccuracy():
    acc = 0
    for i, (data, labels) in enumerate(test_loader, 0):
        
        out = model(data.cuda()).argmax(dim=1)
        labels = labels.cuda().argmax(dim=1)
        print(out)
        acc += (out ==labels).sum()
    
    return acc/len(testset) *100

In [32]:
len(testset)

9

In [33]:
train(100)

tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 2, 1], device='cuda:0')
tensor([1, 1, 2, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1], device='cuda:0')
For epoch 1 the test accuracy over the whole test set is 188 %
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1], device='cuda:0')
For epoch 2 the test accuracy over the whole test set is 177 %
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor

tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1], device='cuda:0')
For epoch 20 the test accuracy over the whole test set is 177 %
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 2, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1], device='cuda:0')
For epoch 21 the test accuracy over the whole test set is 177 %


Exception ignored in: <function _releaseLock at 0x7fa38f1a87a0>
Traceback (most recent call last):
  File "/home/dcvl/anaconda3/envs/gphmr/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt


RuntimeError: DataLoader worker (pid(s) 27876) exited unexpectedly

In [9]:
model(data.cuda())

tensor([[-0.5259,  0.4290,  0.0795, -0.2652],
        [-0.5259,  0.4290,  0.0795, -0.2652],
        [-0.5259,  0.4290,  0.0795, -0.2652],
        [-0.5259,  0.4290,  0.0795, -0.2652]], device='cuda:0',
       grad_fn=<AddmmBackward0>)